# Setup / Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

# Split the ebook into proper chapters

In [2]:
from ebooklib import epub

file1 = "../data/frankenstein.epub"
book1 = epub.read_epub(file1, options={'ignore_ncx': False})

/Users/jimmyyao/miniconda3/envs/novel/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/Users/jimmyyao/miniconda3/envs/novel/lib/python3.11/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [3]:
from ebook_tree_parser.toctree import TocTree

In [4]:
tree = TocTree(book1)
tree

TOC Tree: title=root, href=#, level=0, #children=32

# Prepare the simple output folder structure

In [23]:
# make directory for the output, if it exists, create a new one with incremented number
import os
import shutil

output_dir = "../output/frankenstein"
if os.path.exists(output_dir):
    i = 1
    while os.path.exists(output_dir + "_" + str(i)):
        i += 1
    output_dir += "_" + str(i)
os.makedirs(output_dir)

# make folders "inprompt", "json", and "response" folders in the output directory
os.makedirs(os.path.join(output_dir, "inprompt"))
os.makedirs(os.path.join(output_dir, "json"))
os.makedirs(os.path.join(output_dir, "response"))

# Import openai and get the API key

In [24]:
from openai import OpenAI

from tiktoken import TikToken
# count number of tokens in text
def count_tokens(text):
    return len(TikToken.tokenize(text))

In [20]:
from novelinsights.prompts.fiction import template

print(template(tree[0].content))

# Task
Your task is to create a json according to the following json schema to help readers quickly access information about a novel. 
The novel is a work of fiction and can be any genre.
The information you provide should be rich and detailed, covering various aspects of the novel such as characters, plot points, world-building, themes, and more.
But remember, the information should be concise and to the point, avoiding unnecessary details or spoilers.

## JSON Schema
```
{"type": "object", "properties": {"novelInfo": {"type": "object", "description": "Basic information about the novel", "properties": {"title": {"type": "string", "description": "The full title of the novel"}, "author": {"type": "string", "description": "The name of the author(s)"}, "genre": {"type": "array", "items": {"type": "string"}, "description": "List of genres the novel belongs to"}, "lastUpdated": {"type": "string", "format": "date-time", "description": "Timestamp of the last update to this data"}}, "required"